In our human annnotations we found severe disagreement between annotators. In this file
- I perform some preliminary explorations on the nature of this disagreement
- I run GPT-4 labelling on all other samples
- I compare the classifications of GPT-3 with the other users

#### Explorations

In [1]:
import pandas as pd
import json

The files in this repository
1. new_annotated_sample.pkl: the ~1200 samples with labels generated by us
2. annotated_test.pkl: exactly the same
3. annotated_val.pkl: labels by law students
4. disagreement_sample.json: the 50 overlapping samples, where almost 50% disagreement is

In [2]:
f = open('disagreement_sample.json')
data = json.load(f)

In [3]:
labels = [sample["label"] for sample in data]
sums = [sum(sample) for sample in labels]

for i in set(sums):
    print(f"{i} occurs {sums.count(i)} times")

1 occurs 9 times
2 occurs 6 times
3 occurs 4 times
4 occurs 5 times


In [8]:
# finding out how often each labeller assigned label 1
# all around 50%, except labeller 2
labels = pd.DataFrame(labels)
(labels.sum(axis=0)/len(labels)).round(2)

0    0.50
1    0.08
2    0.54
3    0.54
4    0.54
dtype: float64

#### Obtain explanations for entire dataset samples

In [37]:
df = pd.read_pickle("new_annotated_sample.pkl")
df2 = pd.read_pickle("annotated_test.pkl")
df3 = pd.read_pickle("annotated_val.pkl")

In [2]:
# imports
import openai
import sys
sys.path.append('../99-other-project')
from gptClassifier2 import gptclassifier, standard_message

openai.api_key = "" # insert your api key

In [19]:
# execute classification
completions=[]
results = gptclassifier(df, standard_message, completions, timer_frequency=5)

Done!


In [3]:
import pandas as pd
df = pd.read_pickle("data/annotated_val.pkl")
df2 = pd.read_pickle("data/annotated_val_explanaitons_gpt3.pkl")

In [31]:
pd.to_pickle(df,"annotated_val_explantions_gpt3.pkl")

#### comparing GPT-3 agreement

In [1]:
import json
import pandas as pd

In [72]:
# User classifications
with open("./data/full_agreement_sample.json") as f:
    df = json.load(f)
    f.close()

# Obtain GPT-3 classifications
df2 = pd.read_pickle("./data/annotated_test_explantions_gpt3.pkl")

In [83]:
df3 = pd.DataFrame(pd.DataFrame(df)['label'].tolist(), columns=['col_1', 'col_2', 'col_3', 'col_4', 'col_5'])
df = pd.concat([pd.DataFrame(df), df3], axis=1)

In [86]:
df = df.set_index("text")

In [94]:
df2=df2[["caption","explanations-gpt3"]]
df2.set_index("caption", inplace=True)

In [101]:
df = df.join(df2)

In [108]:
df["gpt3_labels"] = [99]*50

In [122]:
for row in df.iterrows():
    if row[1]["explanations-gpt3"].endswith("Potentially sponsored."):
        row[1]["gpt3-labels"] = 1
    else:
        row[1]["gpt3-labels"] = 0

In [143]:
df['gpt3_labels'] = df.apply(lambda row: 1 if row['explanations-gpt3'].endswith("Potentially sponsored.") else 0, axis=1)

In [146]:
df.head(1)

,label,col_1,col_2,col_3,col_4,col_5,explanations-gpt3,gpt3_labels
text,,,,,,,,
"Lilian 🌼 Wide, rose gold-colored stripes alternate with paler peach, in a gorgeous pattern that’ll take you all the way from winter to spring. We have to include a stripe! Earthy dreamy goodness! Lilian, you’re absolutely beautiful! 😍\n⠀⠀⠀⠀⠀⠀⠀⠀⠀\nWhich ring color is your favorite? Comment yours below and we will send this beauty your way! ENTRIES CLOSED CONGRATULATIONS @kelsey_herp please email hello@wildbird.co ❤️\n⠀⠀⠀⠀⠀⠀⠀⠀⠀\n#giveaway #wildbirdgiveaway #ringslinggiveaway #babywearing","[1, 0, 0, 0, 1]",1,0,0,0,1,"Key indicators: '#giveaway', '#wildbirdgiveawa...",1


In [158]:
sum(df["gpt3_labels"]==df["majority"])

37

In [153]:
df['majority'] = df.apply(lambda row: 1 if sum(row['label'])>2 else 0, axis=1)

In [157]:
sum(df["gpt3_labels"])

26

In [22]:
explanations = list(df2[df2["caption"].isin(samples)]["explanations-gpt3"])

In [ ]:
's'.ends

In [30]:
labels = []
for explanation in explanations:
    if explanation.endswith("Potentially sponsored."):
        labels.append(1)
    elif explanation.endswith("not sponsored."):
        labels.append(0)
    elif explanation.endswith("biguous."):
        labels.append("Ambigous")
    else:
        labels.append(explanation)

In [33]:
user_labels = [sample["label"] for sample in df]

In [34]:
user_labels

[[1, 0, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 1, 1],
 [0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 0, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1],
 [1, 0, 1, 1, 1],
 [0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 1, 0],
 [1, 1, 1, 1, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0],
 [1, 1, 0, 1, 1],
 [1, 0, 1, 1, 1],
 [1, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1],
 [1, 0, 1, 0, 1],
 [1, 1, 1, 1, 1],
 [0, 0, 0, 1, 0],
 [1, 0, 1, 0, 1]]

In [37]:
labels_modified0 = labels

In [53]:
user_labels = pd.DataFrame(user_labels)

In [56]:
user_labels[5] = labels_modified0

In [70]:
user_labels

,0,1,2,3,4,5
0,1,0,0,0,1,1
1,0,0,0,0,0,0
2,1,1,1,1,1,1
3,1,1,1,1,1,0
4,0,0,0,0,0,0
5,1,0,0,1,1,1
6,0,0,0,0,0,1
7,1,0,1,0,1,0
8,1,1,1,1,1,0
9,1,1,1,1,1,0


In [27]:
explanations

["Key indicators: '#YoungRock', 'NBC', '@SevenBucksProd'.\nReasoning: This post promotes a TV show. It again branding of the TV Studio @SevenBucksProd indicates commercial ties. Label: Potentially sponsored.",
 'Reasoning: No indication of promoted product or business. Likely not sponsored. Label: Likely not sponsored.',
 "Key indicators: '#Ad', '@hanes', '@target', '@shop.LTK'.\nReasoning: The caption starts with '#Ad', indicating it is indeed an advertisement. There are two brands featured, @hanes and @target. The post also explicitly calls for a shopping action, making it certainly an advertisement. Label: Potentially sponsored.",
 'Reasoning: The post focuses on a personal story about kayaking and the skill of staying calm in changing situations. There are no apparent indicators of promoting a brand or business. Likely not sponsored.',
 'Reasoning: No indication of sponsoring or promotion. Likely not sponsored. Label: Likely not sponsored.',
 "Key indicators: 'wearing dress by @joa

In [4]:
import pandas as pd
df = pd.read_pickle("./data/annotated_test_explantions_gpt3.pkl")

In [9]:
for element in df.iloc[-30:-25,-1]:
    print(element)

Key indicators: '@byegreis'.
Reasoning: The post promotes clothing brand @byegreis. Clearly self advertisement. Label: Self advertisement.
Key indicators: '@bulgari'.
Reasoning: The post mentions and tags  @bulgari, which could be an indication of a paid partnership. Label: Potentially sponsored.
Key indicators: '@rico5uave', '#RicaSuave'.
Reasoning: The post is mainly about the author and her activity. While it seems to refer to an artwork or a person @rico5uave, this reference is hardly evidenced to bear any commercial or sponsoring element. Not sponsored. Label: Likely not sponsored.
Key indicators: '@FashionNova Ambassador'.
Reasoning: The post explicitly mentions to be a Fashion Nova Ambassador. It clearly is sponsored. Label: Potentially sponsored.
Key indicators: '@pinklily', 'LTK'.
Reasoning: The post promotes a fashion brand and features a discount code, indicating a partnership. Additionally it features an @shop.LTK link, a platform for paid partnerships. Label: Potentially s

In [3]:
import sys
import pandas as pd
sys.path.append('../7-Self-Labelled-Data/data/')
df = pd.read_pickle("../7-Self-Labelled-Data/data/annotated_val_explantions_gpt3.pkl")

### Exploring GPT-label disagreements

In [2]:
import pandas as pd
df=pd.read_pickle("data/annotated_test_explantions_gpt3.pkl")

In [3]:
gpt_labels = pd.Series([True if ((prediction.endswith('ly sponsored.')) or 
                                 (prediction.endswith('ly sponsored')) or 
                                 (prediction.endswith("ly sponsored'.")) or 
                                 (prediction.endswith('ly Sponsored.')) or 
                                 (prediction.endswith(": Sponsored."))) else False if (
                                 (prediction.endswith('not sponsored.')) or 
                                 (prediction.endswith('not sponsored'))) else False if (
                                 (prediction.endswith('biguous.')) or 
                                 (prediction.endswith('biguous'))) else False if (
                                 (prediction.endswith('elf advertisement.')) or 
                                 (prediction.endswith('elf Advertisement.')) or 
                                 (prediction.endswith('isement).')) or 
                                 (prediction.endswith('nal promotion).')) or 
                                 
                                 (prediction.endswith('elf advertisement'))) else prediction
                                 for prediction in df["explanations-gpt3"]])

In [4]:
df.reset_index(inplace=True)
df['gpt_label'] = gpt_labels

In [5]:
disagreements = df[df["gpt_label"] != df["label"]]

In [6]:
disagreements["caption"].to_csv('data/disagreement_captions.csv')

In [11]:
disagreements["explanations-gpt3"].iloc[1]

"Key indicators: 'Follow @wildbirdbaby', 'Tag friends', 'Share to stories'.\nReasoning: The post clearly promotes Wildbird, calling it 'baby brand' and is offering the chance to win some company items. These activities are often effective marketing techniques, therefore it is most likely a self advertisement. Label: Self advertisement."

In [42]:
disagreements.head(20)

,index,username,caption,caption_hashtags,tagged_users,subscriber_count,ct_score,counts_disabled,likes,likes_expected,...,shortcode,followers,dt_year_mon,predicted_disclosure,year,caption_len,label,caption_2,explanations-gpt3,gpt_label
0,43427,therock,"Ahead of TONIGHT’s #YoungRock season premiere,...","[YoungRock, 1, YOUNGROCK]","[8PM, NBC, SevenBucksProd]",303804301,-2.267353,False,267341,605276,...,CbJCKIvFqMI,338004780,2022/03,False,2022,37,False,Ahead of TONIGHT’s HT_YoungRock season premier...,"Key indicators: '#YoungRock', 'NBC', '@SevenBu...",True
2,558102,wildbird,🌟💖 LET’S CELEBRATE WILDBIRD BABY! WIN THE WHOL...,"[babybrand, babyessentials, linenbedding, line...","[brenden_graczak, wildbird, wildbirdbaby]",328731,14.205208,False,4135,933,...,CZ7Pqk8pGHX,326541,2022/02,True,2022,113,True,🌟💖 LET’S CELEBRATE WILDBIRD BABY! WIN THE WHOL...,"Key indicators: 'Follow @wildbirdbaby', 'Tag f...",False
6,778440,thestilettomeup,Back at the office 💛\n\nYellow is my absolute ...,[],"[byegreis, dior, hermes]",513026,1.148215,False,6312,5486,...,Cb8NNmuJ8I6,519012,2022/04,True,2022,21,True,Back at the office 💛\n\nYellow is my absolute ...,"Key indicators: 'Outfit @byegreis', 'Bag @dior...",False
8,778263,thestilettomeup,"I don’t come out often, but when I do….. 😈 \n\...",[],[byegreis],517781,-50.770115,True,0,4348,...,CgK-UTiJfYa,519012,2022/07,False,2022,17,True,"I don’t come out often, but when I do….. 😈 \n\...",Key indicators: 'Suit @byegreis'.\nReasoning: ...,False
13,468500,questlove,Let’s GO LESSSSGO let us GO & do our thang (ou...,[Repost],"[coffeebluffpictures, aetv, versailles73]",2461867,3.305797,False,18628,5609,...,CabKYdwjmxH,2652823,2022/02,True,2022,184,False,Let’s GO LESSSSGO let us GO & do our thang (ou...,"Key indicators: '@aetv', 'Mick Jagger', 'Quest...",True
19,99782,chrisjerichofozzy,Great chat with @fozzyrock talking all about o...,"[Boombox, TalkIsJericho, ChineseFozzocracy, Pu...","[fozzyrock, officialrichward, johnnyandrewsmus...",4093055,-1.898110,False,4545,8652,...,CdZPHZ7Oihh,4217971,2022/05,True,2022,165,False,Great chat with USR_fozzyrock talking all abou...,Key indicators: 'talking all about our new rec...,True
28,946130,richcmanley,Slipping through the river like an elusive ser...,"[riverrun, kernriver, kayaklife, whitewaterkay...","[aquabound, jackson, astralfootwear, sweetprot...",281800,-1.639702,False,5384,8943,...,CdBgFUcpwpc,291227,2022/05,True,2022,160,True,Slipping through the river like an elusive ser...,Reasoning: The post focuses on a personal stor...,False
31,668490,modsun,“Remember Me Just Like This” a 2 hour document...,[],"[rodtheplanter, bignoise]",710107,1.029486,False,20256,20117,...,CZ5Jdy_g91I,745498,2022/02,True,2022,49,False,“Remember Me Just Like This” a 2 hour document...,"Key indicators: 'A film by @rodtheplanter', 'P...",True
36,44196,therock,Blown away by the thunderous response I was lu...,"[1, CinemaCon, DCLeagueOfSuperPets, BLACKADAM,...","[cinemacon, sevenbucksprod, wbpictures]",312987604,-4.795059,False,312851,1499929,...,Cc9NldmJt8B,338004780,2022/04,True,2022,132,False,Blown away by the thunderous response I was lu...,"Key indicators: '@sevenbucksprod', 'DC LEAGUE ...",True
40,220834,weworewhat,"When I tell you, the new active is 🔥🔥🔥 [and ev...",[],[],2960623,-2.483537,False,3639,9071,...,CgXIvpUu6yx,3009747,2022/07,True,2022,21,True,"When I tell you, the new active is 🔥🔥🔥 [and ev...","Key indicators: 'new active', '@weworewhat '.\...",False


In [48]:
no_ad = df[(df["gpt_label"]==False) & (df["label"]==False)]["caption"]
no_ad.to_csv("data/no_ad.csv")